In [13]:
import numpy as np
import sys
from import_lib.func import sphere, ackley, rosenbrock
from import_lib.GA import population_init, factorial_cost, factorial_rank\
                            ,polynomial_mutation, sbx_crossover

In [14]:
tasks =    [sphere(30, limited_space= True, lower_bound= -100, upper_bound= 100), 
            ackley(50, limited_space= True, lower_bound= -100, upper_bound= 100), 
            rosenbrock(20, limited_space= True, lower_bound= -100, upper_bound= 100), 
            rosenbrock(30, limited_space= True, lower_bound= -100, upper_bound= 100), 
            sphere(20, 200, limited_space= True, lower_bound= -300, upper_bound= 300)
            ]

In [15]:
# FIXME
# MFEA1 ???
def MFEA1(num_epochs, num_inds_each_task, range_init_pop = [0, 1],tasks = [], rmp = 0.3, nc = 15, nm = 15, mutate = 'polynomial',
                one_line = False, num_epochs_printed = 20, let_p_be_alive:bool = False):
    
    #save history of factorial cost
    history_cost = np.empty((0, len(tasks)), np.float)
    
    max_d = 0
    for t in tasks:
        if max_d < t.d:
            max_d = t.d
    
    # initial population and skill_factor_arr
    inf, sup = range_init_pop
    population, skill_factor_arr = population_init(
        N = num_inds_each_task, 
        num_tasks = len(tasks), 
        d = max_d,
        min_val = inf,
        max_val = sup,
    )
    
    pop_fcost = factorial_cost(population, skill_factor_arr, tasks)

    for epoch in range(num_epochs):
        # selection
        pop_finess = 1/factorial_rank(pop_fcost, skill_factor_arr, len(tasks))
        num_inds_pop = np.int(num_inds_each_task/2)* len(tasks)
        selected_inds = np.argsort(-pop_finess)[:num_inds_pop] 
        population, skill_factor_arr = population[selected_inds], skill_factor_arr[selected_inds]

        #initial offspring for epoch
        offspring = np.empty((0, max_d))
        offspring_skill_factor = np.empty((0, 1), np.int)

        #nums offspring generated for each task
        num_offspring_each_task = np.zeros(len(tasks), np.int)

        # if next_gen = offspring + parent
        if let_p_be_alive == True:
            offspring = population
            offspring_skill_factor = skill_factor_arr

            for i in skill_factor_arr:
                num_offspring_each_task[i] += 1

        while np.all(num_offspring_each_task < num_inds_each_task):
            [idx_a, idx_b] = np.random.randint(num_inds_pop, size = 2)
            [pa, pb], [skf_a, skf_b] = population[[idx_a, idx_b]], skill_factor_arr[[idx_a, idx_b]]

            if skf_a == skf_b:
                # intra - crossover
                ca, cb = sbx_crossover(pa, pb, nc)

                offspring = np.append(offspring, [ca, cb], axis = 0)
                offspring_skill_factor = np.append(offspring_skill_factor, [skf_a, skf_a])
                num_offspring_each_task[skf_a] += 2
            elif np.random.uniform() <= rmp:
                # inter - crossover
                ca, cb = sbx_crossover(pa, pb, nc)

                offspring = np.append(offspring, [ca, cb], axis = 0)
                skf_ca, skf_cb = np.random.choice([skf_a, skf_b], 2, True)
                offspring_skill_factor = np.append(offspring_skill_factor, [skf_ca, skf_cb])
                num_offspring_each_task[skf_ca] += 1
                num_offspring_each_task[skf_cb] += 1
            else:
                # mutation
                ca, cb = polynomial_mutation(pa, nm, all_dimensions= True), polynomial_mutation(pb, nm, all_dimensions= True)

                
                offspring = np.append(offspring, [ca, cb], axis = 0)
                offspring_skill_factor = np.append(offspring_skill_factor, [skf_a, skf_b])   
                num_offspring_each_task[skf_a] += 1
                num_offspring_each_task[skf_b] += 1      
        
        #update population, skill factor, factorial cost
        population, skill_factor_arr = offspring, offspring_skill_factor
        pop_fcost = factorial_cost(population, skill_factor_arr, tasks)

        #save history
        history_cost = np.append(history_cost, [[np.min(pop_fcost[np.where(skill_factor_arr == idx)[0]]) for idx in range (len(tasks))]], axis = 0)

        #print
        if (epoch + 1) % (num_epochs // num_epochs_printed) == 0:
            if one_line == True:
                sys.stdout.write('\r')
            sys.stdout.write('Epoch [{}/{}], [%-20s] %3d%% ,func_val: {}'
                .format(epoch + 1, num_epochs,[np.min(pop_fcost[np.where(skill_factor_arr == idx)[0]]) for idx in range (len(tasks))])
                % ('=' * ((epoch + 1) // (num_epochs // 20)) + '>' , (epoch + 1) * 100 // num_epochs)
                )
            if one_line == False:
                print("\n")
            sys.stdout.flush()
    print('END!')
    #find solve
    sol_idx = [np.argmin(pop_fcost[np.where(skill_factor_arr == idx)]) for idx in range (len(tasks))]
    sol = [task.decode(population[np.where(skill_factor_arr == idx)][sol_idx[idx]]) for idx, task in enumerate(tasks)]

    return sol, history_cost

In [16]:
sol, his = MFEA1(num_epochs= 2000, num_inds_each_task= 100, range_init_pop= [0, 1],tasks = tasks, rmp = 0.3, nc = 15, nm = 15, one_line=True)


Epoch [2000/2000], [====================>] 100% ,func_val: [0.0038062681222846566, 0.37396658388120274, 2.794908867146226, 2.071190464305557, 4.2194011021114735]END!
